<a href="https://colab.research.google.com/github/shinjangwoon/Segmentation/blob/master/AIDirect_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/Shareddrives/segmentation/train_dataset(update)/train_dataset/

/content/drive/Shareddrives/segmentation/train_dataset(update)/train_dataset


In [3]:
ls

balloon/                          download.sh       train_mask/
balloon_dataset.zip               environment.yaml  val2014/
convert.py                        __MACOSX/         val2014.zip
convert.sh                        requirements.txt  val2014.zip.1
densepose_coco_2014_minival.json  train2014/        val_mask/
densepose_coco_2014_train.json    train2014.zip
Dockerfile                        train2014.zip.1


In [ ]:
!bash download.sh 


Download Densepose coco json
--2021-11-03 14:14:55--  https://dl.fbaipublicfiles.com/densepose/densepose_coco_2014_train.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 551127946 (526M) [text/plain]
Saving to: ‘densepose_coco_2014_train.json.1’

densepose_coco_2014 100%[===================>] 525.60M  16.1MB/s    in 33s     

2021-11-03 14:15:28 (16.0 MB/s) - ‘densepose_coco_2014_train.json.1’ saved [551127946/551127946]

--2021-11-03 14:15:28--  https://dl.fbaipublicfiles.com/densepose/densepose_coco_2014_minival.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31513391

In [5]:
!pip install -r requirements.txt

Fatal Python error: initsite: Failed to import the site module
Traceback (most recent call last):
  File "/usr/lib/python3.7/site.py", line 596, in <module>
    main()
  File "/usr/lib/python3.7/site.py", line 583, in main
    known_paths = addsitepackages(known_paths)
  File "/usr/lib/python3.7/site.py", line 366, in addsitepackages
    addsitedir(sitedir, known_paths)
  File "/usr/lib/python3.7/site.py", line 208, in addsitedir
    names = os.listdir(sitedir)
KeyboardInterrupt


In [ ]:
!bash convert.sh

Extract Train data Segmentation Mask
loading annotations into memory...


In [ ]:
ls

In [ ]:
from sklearn.model_selection import train_test_split
from glob import glob

train_image = glob('./train2014/*.jpg')
train_mask = glob('./train_mask/*.png')

In [ ]:
valid_image = glob('./val2014/*.jpg')
valid_mask = glob('./val_mask/*.png')

print(len(train_image), len(train_mask), len(valid_image), len(valid_mask))

#Img load test

In [ ]:
ls

In [ ]:
jsonTrain = glob('./val2014/densepose_coco_2014_minival.json')
print(jsonTrain)

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
dataDir='./val2014'
dataType='densepose_coco_2014_minival'
annFile='{}/{}.json'.format(dataDir,dataType)
print(annFile)

In [ ]:
coco=COCO(annFile)

In [ ]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [ ]:
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds );
imgIds = coco.getImgIds(imgIds = [458755])
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]

In [ ]:
I = io.imread('%s/%s'%(dataDir,img['file_name']))
plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

#Model test

In [ ]:
cd /content/drive/Shareddrives/segmentation/train_dataset(update)/train_dataset

In [ ]:
!unzip balloon_dataset.zip

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog
import os
import json

def get_balloon_dicts(img_dir):
    json_file = os.path.join(img_dir, 'via_region_data.json')
    with open(json_file) as f:
        imgs_anns = json.load(f)
    
    dataset_dixts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}

        filename = os.path.join(img_dir, v['filename'])
        height, width = cv2.imread(filename).shape[:2]

        record['filename'] = filename
        record['image_id'] = idx
        record['height'] = height
        record['width'] = width

        annos = v['regions']
        objs = []

        for _, anno in annos.items():
            anno = anno['shape_attributes']
            px = anno['all_points_x']
            py = anno['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                'bbox' : [np.min(px), np.min(py), np.max(px), np.max(py)],
                'bbox_mode' : BoxMode.XYXY_ABS,
                'segmentation' : [poly],
                'category_id' : 0,
            }
            objs.append(obj)
        
        record['annotations'] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ['train', 'val']:
    DatasetCatalog.register('balloon_' + d, lambda d=d: get_balloon_dicts('balloon/' + d))
    MetadataCatalog.get('balloon_' + d).set(thing_classes=['balloon'])

balloon_metadata = MetadataCatalog.get('balloon_train')

In [ ]:
import cv2, random
import numpy as np

dataset_dicts = get_balloon_dicts('balloon/train')

for d in random.sample(dataset_dixts, 3):
    img = cv2.imread(d['file_name'])
    visualizer = visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])